In [0]:
# Start a new PySpark session with the following code and show the Iris dataset: 
import pyspark
from pyspark.sql import SparkSession
# Start Spark Session
spark = SparkSession.builder.appName("Operations").getOrCreate()

In [0]:
# Load data from the Iris dataset
df1=sqlContext.sql("select * from default.iris_data")
df1.show(10)

+------------+-----------+------------+-----------+-----------+
|Sepal_Length|Sepal_Width|Petal_Length|Petal_Width|      Class|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|
+------------+-----------+------------+-----------+-----------+
only showing top 10 rows



In [0]:
# 2.	Load any dataset of your choosing into another new Dataframe called ‘df2’

df2=sqlContext.sql("select * from default.annual_change_forest_area")
df2.show(10)

+---------+----+----+--------------------------------+
|   Entity|Code|Year|Annual_Net_Change_In_Forest_Area|
+---------+----+----+--------------------------------+
|  Algeria| DZA|1990|                         -8800.0|
|  Algeria| DZA|2000|                         33900.0|
|  Algeria| DZA|2010|                          7600.0|
|  Algeria| DZA|2015|             -1400.0000000000005|
|Argentina| ARG|1990|                       -182600.0|
|Argentina| ARG|2000|                       -316400.0|
|Argentina| ARG|2010|                       -223400.0|
|Argentina| ARG|2015|                       -104800.0|
|    Aruba| ABW|1990|                             0.0|
|    Aruba| ABW|2000|                             0.0|
+---------+----+----+--------------------------------+
only showing top 10 rows



In [0]:
# 3.	Display the column names of both your dataframes using the columns command
print(f'Column names for df1: {df1.columns}')
print(f'Column names for df2: {df2.columns}')

Column names for df1: ['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width', 'Class']
Column names for df2: ['Entity', 'Code', 'Year', 'Annual_Net_Change_In_Forest_Area']


In [0]:
# Display the schema for both your dataframes using the printSchema function
print(f'Schema for df1: {df1.describe().printSchema()}')
print(f'Schema for df2: {df2.describe().printSchema()}')

root
 |-- summary: string (nullable = true)
 |-- Sepal_Length: string (nullable = true)
 |-- Sepal_Width: string (nullable = true)
 |-- Petal_Length: string (nullable = true)
 |-- Petal_Width: string (nullable = true)
 |-- Class: string (nullable = true)

Schema for df1: None
root
 |-- summary: string (nullable = true)
 |-- Entity: string (nullable = true)
 |-- Code: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Annual_Net_Change_In_Forest_Area: string (nullable = true)

Schema for df2: None


In [0]:
# 4. Learn about your dataframes using the describe function to display descriptive statistics
df1.describe().show()
df2.describe().show()

+-------+------------------+-------------------+------------------+------------------+--------------+
|summary|      Sepal_Length|        Sepal_Width|      Petal_Length|       Petal_Width|         Class|
+-------+------------------+-------------------+------------------+------------------+--------------+
|  count|               150|                150|               150|               150|           150|
|   mean| 5.843333333333335| 3.0540000000000007|3.7586666666666693|1.1986666666666672|          null|
| stddev|0.8280661279778637|0.43359431136217375| 1.764420419952262|0.7631607417008414|          null|
|    min|               4.3|                2.0|               1.0|               0.1|   Iris-setosa|
|    max|               7.9|                4.4|               6.9|               2.5|Iris-virginica|
+-------+------------------+-------------------+------------------+------------------+--------------+

+-------+--------+----+------------------+--------------------------------+
|summ

In [0]:
# The summary() function looks like an extension of the describe function.

df1.summary().show()

+-------+------------------+-------------------+------------------+------------------+--------------+
|summary|      Sepal_Length|        Sepal_Width|      Petal_Length|       Petal_Width|         Class|
+-------+------------------+-------------------+------------------+------------------+--------------+
|  count|               150|                150|               150|               150|           150|
|   mean| 5.843333333333335| 3.0540000000000007|3.7586666666666693|1.1986666666666672|          null|
| stddev|0.8280661279778637|0.43359431136217375| 1.764420419952262|0.7631607417008414|          null|
|    min|               4.3|                2.0|               1.0|               0.1|   Iris-setosa|
|    25%|               5.1|                2.8|               1.6|               0.3|          null|
|    50%|               5.8|                3.0|               4.3|               1.3|          null|
|    75%|               6.4|                3.3|               5.1|               

In [0]:
# 5.	Use a filter function to find data in the Iris dataframe df1, where SepalLength is less than 5 and display data showing all columns -
df1.filter(df1.Sepal_Length < 5).show()

+------------+-----------+------------+-----------+-----------+
|Sepal_Length|Sepal_Width|Petal_Length|Petal_Width|      Class|
+------------+-----------+------------+-----------+-----------+
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|
|         4.8|        3.4|         1.6|        0.2|Iris-setosa|
|         4.8|        3.0|         1.4|        0.1|Iris-setosa|
|         4.3|        3.0|         1.1|        0.1|Iris-setosa|
|         4.6|        3.6|         1.0|        0.2|Iris-setosa|
|         4.8|        3.4|         1.9|        0.2|Iris-setosa|
|         4.7|        3.2|         1.6|        0.2|Iris-setosa|
|         4.8|        3.1|         1.6| 

In [0]:
# 5. and create a similar filter based on your df2 dataframe 
df2.filter(df2.Annual_Net_Change_In_Forest_Area > 150000).show()

+-------------+----+----+--------------------------------+
|       Entity|Code|Year|Annual_Net_Change_In_Forest_Area|
+-------------+----+----+--------------------------------+
|    Australia| AUS|2010|               709680.0000000002|
|        Chile| CHL|2010|                        174160.0|
|        China| CHN|1990|                       1986000.0|
|        China| CHN|2000|                       2360980.0|
|        China| CHN|2010|                       1936770.0|
|        China| CHN|2015|                       1936790.0|
|        India| IND|1990|                        365300.0|
|        India| IND|2000|                        190500.0|
|        India| IND|2010|                        266400.0|
|        India| IND|2015|                        266400.0|
|       Russia| RUS|2000|                        586710.0|
|United States| USA|2000|                        518400.0|
|United States| USA|2010|                        275000.0|
|      Vietnam| VNM|1990|                        240810.

In [0]:
# 6.	Use a filter function to find data in the Iris dataframe df1 where 
#       SepalWidth is exactly 4 and display data showing all columns
df1.filter(df1.Sepal_Width == 4.0).show()

+------------+-----------+------------+-----------+-----------+
|Sepal_Length|Sepal_Width|Petal_Length|Petal_Width|      Class|
+------------+-----------+------------+-----------+-----------+
|         5.8|        4.0|         1.2|        0.2|Iris-setosa|
+------------+-----------+------------+-----------+-----------+



In [0]:
# 6. and create a similar filter based on your df2 dataframe 
df2.filter(df2.Code == 'IRL').show()

+-------+----+----+--------------------------------+
| Entity|Code|Year|Annual_Net_Change_In_Forest_Area|
+-------+----+----+--------------------------------+
|Ireland| IRL|1990|              16869.999999999993|
|Ireland| IRL|2000|                          9000.0|
|Ireland| IRL|2010|                          6860.0|
|Ireland| IRL|2015|                          5470.0|
+-------+----+----+--------------------------------+



In [0]:
# 7.    Use a filter function to find data in the Iris dataframe df1, where SepalLength
2
#       is greater than 5 and display only the SepalLength & Class columns 
3
df1.filter("Sepal_Length > 5").select('Sepal_Length','Class').show()

+------------+-----------+
|Sepal_Length|      Class|
+------------+-----------+
|         5.1|Iris-setosa|
|         5.4|Iris-setosa|
|         5.4|Iris-setosa|
|         5.8|Iris-setosa|
|         5.7|Iris-setosa|
|         5.4|Iris-setosa|
|         5.1|Iris-setosa|
|         5.7|Iris-setosa|
|         5.1|Iris-setosa|
|         5.4|Iris-setosa|
|         5.1|Iris-setosa|
|         5.1|Iris-setosa|
|         5.2|Iris-setosa|
|         5.2|Iris-setosa|
|         5.4|Iris-setosa|
|         5.2|Iris-setosa|
|         5.5|Iris-setosa|
|         5.5|Iris-setosa|
|         5.1|Iris-setosa|
|         5.1|Iris-setosa|
+------------+-----------+
only showing top 20 rows



In [0]:
# 7. - and create a similar filter based on your df2 dataframe
df2.filter("Annual_Net_Change_In_Forest_Area < -1726500").select('Entity','Annual_Net_Change_In_Forest_Area','Year').show()

+------+--------------------------------+----+
|Entity|Annual_Net_Change_In_Forest_Area|Year|
+------+--------------------------------+----+
|Brazil|                      -3780940.0|1990|
|Brazil|                      -3950790.0|2000|
| World|                      -7818000.0|1990|
| World|                      -5117000.0|2000|
| World|                      -4801000.0|2010|
| World|                      -5150000.0|2015|
+------+--------------------------------+----+



In [0]:
# 8.	Use a filter function to find data in the Iris dataframe df1, where SepalLength is greater 
#       than 3 AND SepalWidth is greater than 4 and display data showing all columns   

df1.filter((df1['Sepal_Length'] > 3) & (df1['Sepal_Width'] > 4)).show()

+------------+-----------+------------+-----------+-----------+
|Sepal_Length|Sepal_Width|Petal_Length|Petal_Width|      Class|
+------------+-----------+------------+-----------+-----------+
|         5.7|        4.4|         1.5|        0.4|Iris-setosa|
|         5.2|        4.1|         1.5|        0.1|Iris-setosa|
|         5.5|        4.2|         1.4|        0.2|Iris-setosa|
+------------+-----------+------------+-----------+-----------+



In [0]:
# 8.	- and create a similar filter based on your df2 dataframe

df2.filter((df2['Year'] > 1990) & (df2['Annual_Net_Change_In_Forest_Area'] < -750000)).show()

+------+--------+----+--------------------------------+
|Entity|    Code|Year|Annual_Net_Change_In_Forest_Area|
+------+--------+----+--------------------------------+
|Brazil|     BRA|2000|                      -3950790.0|
|Brazil|     BRA|2010|             -1539179.9999999995|
|Brazil|     BRA|2015|                      -1453040.0|
| World|OWID_WRL|2000|                      -5117000.0|
| World|OWID_WRL|2010|                      -4801000.0|
| World|OWID_WRL|2015|                      -5150000.0|
+------+--------+----+--------------------------------+



In [0]:
# 9.	Use the groupBy function to calculate the highest value for each field/column in both dataframes - grouped by Class in df1: 

df1.groupBy("Class").max("Sepal_Length","Sepal_Width","Petal_Length","Petal_Width").show(truncate=False)

+---------------+-----------------+----------------+-----------------+----------------+
|Class          |max(Sepal_Length)|max(Sepal_Width)|max(Petal_Length)|max(Petal_Width)|
+---------------+-----------------+----------------+-----------------+----------------+
|Iris-virginica |7.9              |3.8             |6.9              |2.5             |
|Iris-setosa    |5.8              |4.4             |1.9              |0.6             |
|Iris-versicolor|7.0              |3.4             |5.1              |1.8             |
+---------------+-----------------+----------------+-----------------+----------------+



In [0]:
# 9. , and a column of your choosing in df2

df2.groupBy("Entity").max("Annual_Net_Change_In_Forest_Area","Year").show(truncate=False)

+-----------------+-------------------------------------+---------+
|Entity           |max(Annual_Net_Change_In_Forest_Area)|max(Year)|
+-----------------+-------------------------------------+---------+
|Paraguay         |-255420.0                            |2015     |
|Russia           |586710.0                             |2015     |
|World            |-4801000.0                           |2015     |
|Senegal          |-40000.0                             |2015     |
|Sweden           |9999.999999999998                    |2015     |
|Tokelau          |0.0                                  |2015     |
|Guyana           |-3790.0                              |2015     |
|Djibouti         |40.000000000000014                   |2015     |
|Singapore        |220.0                                |2015     |
|Turkey           |118010.0                             |2015     |
|Malawi           |-42000.0                             |2015     |
|Iraq             |1400.0                       

In [0]:
# 10.	Calculate the Standard Deviation for SepalWidth and format the result to show 2 decimal places in Iris df1
from pyspark.sql.functions import stddev,round
df1.select(stddev("Sepal_Width"),round(stddev('Sepal_Width'),2)).show()

+------------------------+----------------------------------+
|stddev_samp(Sepal_Width)|round(stddev_samp(Sepal_Width), 2)|
+------------------------+----------------------------------+
|     0.43359431136217375|                              0.43|
+------------------------+----------------------------------+



In [0]:
# 10.  and do a similar calculation in df2
from pyspark.sql.functions import round
df2.select(stddev("Annual_Net_Change_In_Forest_Area"),round(stddev('Annual_Net_Change_In_Forest_Area'),2)).show()


+---------------------------------------------+-------------------------------------------------------+
|stddev_samp(Annual_Net_Change_In_Forest_Area)|round(stddev_samp(Annual_Net_Change_In_Forest_Area), 2)|
+---------------------------------------------+-------------------------------------------------------+
|                            639354.3795674981|                                              639354.38|
+---------------------------------------------+-------------------------------------------------------+



In [0]:
## End of Lab 4